### **IMPORTACIONES**

In [43]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
from langchain_ollama import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter

### Variable de entorno

In [44]:
load_dotenv() 

api_key = os.getenv("GOOGLE_API_KEY")

### Subir PDF (PyPDFLoader)

In [45]:
def upload_pdf(file_path):
    try: 
        loader = PyPDFLoader(file_path)
        loader = loader.lazy_load()
        
        text = ""
        for page in loader:
            text += page.page_content + "\n"
        return text
    except Exception as e:
        print(f"Error al cargar el PDF: {e}")
        return []

### Separar contenido del PDF (CharacterTextSplitter)

In [46]:
def text_splitter(text):
    try:
        text_splitter = CharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=150,
            separator="\n",
        )
        texts = text_splitter.create_documents([text])
        return texts
    except Exception as e:
        print(f"Error al separar el texto: {e}")
        return []

### BD vectorial flexible (Chroma)

In [47]:
def get_vector_store(name_collection: str, embedding_model, persist_dir: str): 
    
    vector_store = Chroma(
        collection_name= name_collection,
        embedding_function=embedding_model,
        persist_directory=persist_dir     
    )    
    return vector_store

### Modelo original (nomic-embed-text)

In [48]:
embedding_original = OllamaEmbeddings(
    model = "nomic-embed-text"
)

### Vector Store del modelo original

In [49]:
vs_original = get_vector_store(
    name_collection="langchain",
    embedding_model=embedding_original, 
    persist_dir="./vectorstore"
)

### Búsqueda de contenido relevante según el input ingresado 

In [50]:
def retrieval(input_user: str, vector_store): 
    docs = vector_store.similarity_search(input_user)
    return docs

### Prompt (PromptTemplate)

In [51]:
prompt = PromptTemplate.from_template("""
    Eres un asistente encargado de responder preguntas sobre la energía solar y solo debes contestar si el contexto no está vacio.
    En caso de que no cuentes con la información solicitada responde "No cuento con información sobre eso".
    Utiliza siempre el contexto proporcionado para responder.
    contexto = {contexto}
    pregunta del usuario: {input_user}
""")

### Conexión con el modelo y respuesta final (ChatGoogleGenerativeAI)

In [52]:
def response(input_user: str, contexto: str):
    llm = ChatGoogleGenerativeAI(
    api_key=api_key,
    model="gemini-2.5-flash",
    temperature= 0.7
)

    for chunk in llm.stream(prompt.format(contexto=contexto, input_user=input_user)):
        yield chunk.content

In [53]:
loader = upload_pdf("energia_solar.pdf")
texts = text_splitter(loader)

### Guardar el embedding original

In [54]:
vs_original.add_documents(texts)

['fd81654c-8d34-478b-90bc-23c653b159ae',
 '6cdab405-5402-4d66-89e7-38d19198ae3d',
 'bf495313-f0bc-462b-9aac-1024de4c0eba']

---
## Punto 2: Experimento de Embeddings

In [55]:
embedding_nuevo = OllamaEmbeddings(
    model = "mxbai-embed-large"
)

### Vector Store del modelo nuevo

In [56]:
vs_nuevo = get_vector_store(
    name_collection="experimento_mxbai", 
    embedding_model=embedding_nuevo, 
    persist_dir="./vectorstore_2"
)

### Guardar el nuevo embedding

In [57]:
vs_nuevo.add_documents(texts)

['c4210098-e77c-485e-8269-960ca448b73c',
 'aa757f23-4ba9-4abf-94ab-12fa7baf43fa',
 '55c3b012-e8ef-4fcb-9910-bf754c8c3aa6']

### Comparación `nomic-embed-text` vs `mxbai-embed-large`

In [58]:
input_user = input("Human: ")
print(f"Pregunta: {input_user}")


# 1. Prueba con la BD ORIGINAL (nomic-embed-text)
print("\n--- 1. nomic-embed-text ---\n")
docs_original = retrieval(input_user=input_user, vector_store=vs_original)

print("\n[DOCS RECUPERADOS por 'nomic']:")
print([doc.page_content for doc in docs_original])

for chunk in response(input_user=input_user, contexto=docs_original):
    print(chunk, end=" ", flush=True)

# 2. Prueba con la BD NUEVA (mxbai-embed-large)
print("\n\n--- 2. mxbai-embed-large ---\n")
docs_nuevos = retrieval(input_user=input_user, vector_store=vs_nuevo)

print("\n[DOCS RECUPERADOS por 'mxbai']:")
print([doc.page_content for doc in docs_nuevos])

for chunk in response(input_user=input_user, contexto=docs_nuevos):
    print(chunk, end=" ", flush=True)

Pregunta: ¿Qué es la energía solar?

--- 1. nomic-embed-text ---


[DOCS RECUPERADOS por 'nomic']:
['La energía solar y su impacto ambiental \n \nLa energía solar es una fuente de energía renovable que se obtiene a partir de la \nradiación del sol. Esta forma de energía ha ganado popularidad en todo el mundo \ndebido a su potencial para reducir la dependencia de combustibles fósiles y \ndisminuir las emisiones de gases de efecto invernadero. \n \nLos sistemas de energía solar se dividen principalmente en dos categorías: \nenergía solar fotovoltaica y energía solar térmica. La energía fotovoltaica convierte \nla luz solar directamente en electricidad mediante el uso de células solares, \nmientras que la energía térmica utiliza colectores solares para calentar líquidos, \nque luego pueden ser usados para calefacción o generación de electricidad a \ntravés de turbinas de vapor. \n \nUno de los principales beneficios de la energía solar es su sostenibilidad. A \ndiferencia de los combustib

### Análisis de Resultados

 **Pregunta: "¿Qué es la energía solar?"**

 **`nomic-embed-text`:** Recuperó exitosamente el chunk con la definición.

 **`mxbai-embed-large`:** Falló, recuperando el chunk de la conclusión, que era irrelevante.

**Conclusión:** El modelo `nomic-embed-text` es claramente superior para este documento, demostrando una mejor capacidad de recuperación semántica tanto para la definición principal como para detalles específicos.

---
## Experimento de LLMs (Gemini vs. Llama 3)

### Conexión con Llama 3

In [59]:
def response_llama(input_user: str, contexto: str):
    try:
        llm_local = ChatOllama(
            model="llama3", 
            temperature=0.7
            )
        for chunk in llm_local.stream(prompt.format(contexto=contexto, input_user=input_user)):
            yield chunk.content
            
    except Exception as e:
        print(f"Error al conectar con Llama 3: {e}")
        yield f"[Error] No se pudo conectar con Llama 3: {e}"

### Comparación `Gemini` vs `Llama 3`

In [60]:
docs_nomic = docs_original # Los de 'nomic-embed-text'

print(f"Pregunta: {input_user}\n")
print(f"Contexto (Docs): {[doc.page_content for doc in docs_nomic]}\n")


# --- 1. Prueba con Google Gemini (API) ---
print("--- 1. Respuesta de Google Gemini (gemini-2.5-flash) ---")
for chunk in response(input_user=input_user, contexto=docs_nomic):
    print(chunk, end=" ", flush=True)


# --- 2. Prueba con Llama 3 (Local) ---
print("\n\n--- 2. Respuesta de Llama 3 (local) ---")
for chunk in response_llama(input_user=input_user, contexto=docs_nomic):
    print(chunk, end=" ", flush=True)

Pregunta: ¿Qué es la energía solar?

Contexto (Docs): ['La energía solar y su impacto ambiental \n \nLa energía solar es una fuente de energía renovable que se obtiene a partir de la \nradiación del sol. Esta forma de energía ha ganado popularidad en todo el mundo \ndebido a su potencial para reducir la dependencia de combustibles fósiles y \ndisminuir las emisiones de gases de efecto invernadero. \n \nLos sistemas de energía solar se dividen principalmente en dos categorías: \nenergía solar fotovoltaica y energía solar térmica. La energía fotovoltaica convierte \nla luz solar directamente en electricidad mediante el uso de células solares, \nmientras que la energía térmica utiliza colectores solares para calentar líquidos, \nque luego pueden ser usados para calefacción o generación de electricidad a \ntravés de turbinas de vapor. \n \nUno de los principales beneficios de la energía solar es su sostenibilidad. A \ndiferencia de los combustibles fósiles, la energía solar no produce emis

### Análisis de LLMs (Gemini vs. Llama 3)

Ambos modelos recibieron el mismo contexto de alta calidad (de `nomic-embed-text`) y el mismo prompt.

* **Gemini 2.5 Flash (API):** La respuesta fue muy rápida, concisa y **sintetizada**. Entendió la pregunta y extrajo solo la definición esencial.
* **Llama 3 (Local):** La respuesta fue tardada a pesar de ser local, fue más **extractiva**. En lugar de sintetizar, optó por copiar las primeras dos oraciones del contexto.

**Conclusión:** Para un chatbot que debe sonar natural, prefiero el comportamiento de **Gemini**, ya que sintetiza la información en lugar de simplemente copiarla. Sin embargo, **Llama 3** es una excelente alternativa gratuita y local que también encuentra la respuesta correcta.

---
## Punto 3: Optimización del Separador de Texto

### Experimento 3.1: Optimización de `chunk_size`

Inicialmente, usé `CharacterTextSplitter` con un `chunk_size=7000`. Esto fue un error, ya que el PDF era más pequeño y se creó **un solo chunk**. Esto hacía que el RAG fallara.

Al **modificar el parámetro** a `chunk_size=1000`, funcionó todo bastante bien.

### Utilizando otro Splitter (RecursiveCharacterTextSplitter)

In [61]:
text_splitter_recursivo = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150, 
)

texts_recursivos = text_splitter_recursivo.create_documents([loader])

vs_recursivo = get_vector_store(
    name_collection="experimento_recursive",
    embedding_model=embedding_original,  
    persist_dir="./vectorstore_3"       
)

vs_recursivo.add_documents(texts_recursivos)

['4c963537-14a1-4547-8852-0936d42dfee1',
 'ced15042-c7ac-42fc-a2c6-1a33aecbe353',
 'ec9cb0f0-2bb7-4754-a3ab-b102d746c69b']

### Comparación `CharacterTextSplitter` vs `RecursiveCharacterTextSplitter`

In [62]:
input_user_2 = "¿Cuáles son las desventajas de la energía solar?"
print(f"Pregunta: {input_user_2}\n")

# 1. Prueba con CharacterTextSplitter (de vs_original)
print("--- 1. RESULTADO con CharacterTextSplitter")
docs_originales = retrieval(input_user_2, vs_original) 
print([doc.page_content for doc in docs_originales])


# 2. Prueba con RecursiveCharacterTextSplitter (de vs_recursivo)
print("\n--- 2. RESULTADO con RecursiveCharacterTextSplitter")
docs_recursivos = retrieval(input_user_2, vs_recursivo)
print([doc.page_content for doc in docs_recursivos])

Pregunta: ¿Cuáles son las desventajas de la energía solar?

--- 1. RESULTADO con CharacterTextSplitter
['La energía solar y su impacto ambiental \n \nLa energía solar es una fuente de energía renovable que se obtiene a partir de la \nradiación del sol. Esta forma de energía ha ganado popularidad en todo el mundo \ndebido a su potencial para reducir la dependencia de combustibles fósiles y \ndisminuir las emisiones de gases de efecto invernadero. \n \nLos sistemas de energía solar se dividen principalmente en dos categorías: \nenergía solar fotovoltaica y energía solar térmica. La energía fotovoltaica convierte \nla luz solar directamente en electricidad mediante el uso de células solares, \nmientras que la energía térmica utiliza colectores solares para calentar líquidos, \nque luego pueden ser usados para calefacción o generación de electricidad a \ntravés de turbinas de vapor. \n \nUno de los principales beneficios de la energía solar es su sostenibilidad. A \ndiferencia de los combu

### Análisis de Resultados

* **`CharacterTextSplitter` (Original):** Los `DOCS RECUPERADOS` fueron irrelevantes, solo trajeron la definición de la energía solar. Su método de división simple (por `\n`) no fue lo suficientemente preciso.

* **`RecursiveCharacterTextSplitter` (Experimento):** Los `DOCS RECUPERADOS` incluyeron el chunk específico que contenía la respuesta, hablando de "desafíos ambientales", "fabricación... intensivos" y "gestión del reciclaje".

**Conclusión:** El `RecursiveCharacterTextSplitter` es claramente superior. Su método de división más inteligente (que intenta separar por párrafos, luego por líneas, etc.) creó chunks más coherentes semánticamente.

---
## Punto 4: Gestión de Bases de Datos Vectoriales

### Búsquedas avanzadas con filtrado

In [ ]:
# Usamos 'loader' que ya tiene el texto del PDF
texts_con_metadata = text_splitter_recursivo.create_documents([loader])

# Se divide la primer mitad como 'seccion_A' y la 2da como 'seccion_B'
num_docs = len(texts_con_metadata)
for i, doc in enumerate(texts_con_metadata):
    if i < num_docs / 2:
        doc.metadata["seccion"] = "A"
    else:
        doc.metadata["seccion"] = "B"

vs_filtrado = get_vector_store(
    name_collection="experimento_filtrado",
    embedding_model=embedding_original, 
    persist_dir="./vectorstore_4"
)

vs_filtrado.add_documents(texts_con_metadata)

Documentos con metadatos listos. Ejemplo del primer doc: {'seccion': 'A'}
Ejemplo del último doc: {'seccion': 'B'}


['6a5e6276-da84-4f34-807c-0895d716a385',
 'ca897e04-15b0-49e3-8f04-3f1836807c8e',
 'c4b21a3a-e004-4014-b518-e0d5515af8d2']

In [ ]:
print(f"Pregunta: {input_user_2}\n")

# 1. Búsqueda SIN FILTRO (en todas las secciones)
print("--- 1. Búsqueda SIN FILTRO ---")
docs_sin_filtro = retrieval(input_user_2, vs_filtrado) 
print(f"Docs encontrados: {len(docs_sin_filtro)}")
print([doc.page_content for doc in docs_sin_filtro])
for chunk in response(input_user=input_user_2, contexto=docs_sin_filtro):
    print("\n", chunk, end=" ", flush=True)


# 2. Búsqueda CON FILTRO (solo en 'seccion_B')
print("\n--- 2. Búsqueda CON FILTRO (solo en 'seccion_B') ---")
docs_con_filtro = vs_filtrado.similarity_search(
    input_user_2,
    filter={"seccion": "B"}
)
print(f"Docs encontrados: {len(docs_con_filtro)}")
print([doc.page_content for doc in docs_con_filtro])
for chunk in response(input_user=input_user_2, contexto=docs_con_filtro):
    print("\n", chunk, end=" ", flush=True)


# 3. Búsqueda CON FILTRO (solo en 'seccion_A')
print("\n--- 3. Búsqueda CON FILTRO (solo en 'seccion_A') ---")
docs_filtro_fallido = vs_filtrado.similarity_search(
    input_user_2,
    filter={"seccion": "A"}
)
print(f"Docs encontrados: {len(docs_filtro_fallido)}")
print([doc.page_content for doc in docs_filtro_fallido])
for chunk in response(input_user=input_user_2, contexto=docs_filtro_fallido):
    print("\n", chunk, end=" ", flush=True)

Pregunta: ¿Cuáles son las desventajas de la energía solar?

--- 1. Búsqueda SIN FILTRO ---
Docs encontrados: 4
['La energía solar y su impacto ambiental \n \nLa energía solar es una fuente de energía renovable que se obtiene a partir de la \nradiación del sol. Esta forma de energía ha ganado popularidad en todo el mundo \ndebido a su potencial para reducir la dependencia de combustibles fósiles y \ndisminuir las emisiones de gases de efecto invernadero. \n \nLos sistemas de energía solar se dividen principalmente en dos categorías: \nenergía solar fotovoltaica y energía solar térmica. La energía fotovoltaica convierte \nla luz solar directamente en electricidad mediante el uso de células solares, \nmientras que la energía térmica utiliza colectores solares para calentar líquidos, \nque luego pueden ser usados para calefacción o generación de electricidad a \ntravés de turbinas de vapor. \n \nUno de los principales beneficios de la energía solar es su sostenibilidad. A \ndiferencia de l

### Análisis

* **Persistencia:** Se logró fácilmente usando `persist_directory`.
* **Colecciones:** Se gestionaron múltiples colecciones (`langchain`, `experimento_recursive`, etc.) usando `collection_name`.
* **Filtrado:** La búsqueda con `filter` funcionó perfectamente y me permitió descubrir dónde estaba mi información.
    * Al filtrar por `filter={"seccion": "A"}` (Búsqueda 3), el sistema **encontró** el chunk sobre las "desventajas" y dio la respuesta correcta.
    * Al filtrar por `filter={"seccion": "B"}` (Búsqueda 2), el sistema **no encontró** la respuesta (recuperó chunks irrelevantes de la conclusión) y el LLM respondió "No cuento con información...".
    
**Conclusión:** Esto demuestra que el chunk de "desventajas" se encontraba en la primera mitad del documento (`seccion_A`).

### Otras BD vectoriales compatibles con langchain:

* **LanceDB**: Una base de datos vectorial "serverless" (sin servidor) y local. Es una alternativa moderna a Chroma.

    * **Ventajas**
        - Fácil instalación (pip install lancedb)
        - Permite el "versionado" de los datos, para que puedas rastrear cambios o volver a una versión anterior de tu base de datos.
        - Utiliza un formato de almacenamiento (Apache Arrow) que es muy eficiente.
    
    * **Desventajas**
        - Es un proyecto más reciente que Chroma o FAISS, por lo que la comunidad y los ejemplos pueden ser menos extensos.

* **Pinecone**: Gestionada en la nube. Ideal para chatbots en producción y cualquier sistema RAG que necesite escalar y manejar millones de documentos o usuarios.

    * **Ventajas**
        - Puede manejar miles de millones de vectores sin problemas.
        - No tienes que preocuparte por servidores, mantenimiento o disponibilidad.
        - Tiene filtrado de metadatos muy potente, gestión de colecciones y APIs robustas.

    * **Desventajas**
        - Plan gratuito limitado
        - Depende 100% de la API de Pinecone.    

